In [35]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

import os
import yaml

In [36]:
from kfp.v2.dsl import (Artifact,
						Dataset,
						Input,
						Model,
						Output,
						Metrics,
						ClassificationMetrics,
						component)

In [60]:
def read_params(config_path):
    with open (config_path) as yaml_file:
        config = yaml.safe_load(yaml_file)
    return config

In [85]:
help(kfp.components.load_component_from_file)

Help on function load_component_from_file in module kfp.components._components:

load_component_from_file(filename)
    Loads component from file and creates a task factory function.
    
    Args:
        filename: Path of local file containing the component definition.
    
    Returns:
        A factory function with a strongly-typed signature.
        Once called with the required arguments, the factory constructs a pipeline task instance (ContainerOp).



In [95]:
@component(packages_to_install = ["pandas"],)
def load_data(config_path):
    #print("hello world")
    #import yaml
    kfp.components.load_component_from_file('params.yaml')
    #with open (config_path) as yaml_file:
    #    config = yaml.safe_load(yaml_file)
    #config = read_params(config_path='params.yaml')
    #data_path = config["data_source"]["s3_source"]
    #df=pd.read_csv(data_path, sep=",", encoding="utf-8")
    ##print(df.head())
    #return df


In [96]:
@dsl.pipeline(
   name='Winequality pipeline',
   description='Pipeline to get data, split data, train , save model.'
)
def pipeline_wfq(
    config_path
):
    flow_op1 = load_data(config_path)
    #print(flow_op1)
    #flow_op2 = split_data()
    #flow_op3 = train_evalauate()
     

In [97]:
import requests

HOST = "http://40.70.70.47:8080/"
USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow"

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    cookies=f"authservice_session={session_cookie}",
    namespace=NAMESPACE,
)

In [98]:
arguments = {'config_path':'params.yaml'}
client.create_run_from_pipeline_func(pipeline_wfq, arguments=arguments, 
                                           experiment_name="load_data")

RunPipelineResult(run_id=79e13b4d-1fb0-4f8d-83e9-bfe88e9ecbed)